In [1]:
import os
import tensorflow as tf

# input names
model_name = 'efficientnetv2-l'
#model_name = 'resnet152'
model_folder = '/disk/apps/metrabs/models/metrabs_eff2l_y4/'

# output names
base_fold = "mods/effnet-l/"

bbone_name = os.path.join(base_fold, "bbone")
head_name = os.path.join(base_fold, "metrab_head")
onnx_name = os.path.join(base_fold, "bbone.onnx")

model = tf.saved_model.load(model_folder)

mod_vars = model.crop_model.variables
name_shape = [['/'.join(v.name.split('/')[1:]), v.shape] for v in mod_vars if 'tpu' not in v.name]

2022-01-03 16:02:42.544929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 16:02:42.550918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 16:02:42.551368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 16:02:42.552219: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
import sys
import keras
sys.path.append('/home/tobi/apps/metrabs/metrabs/src')

if model_name.startswith('efficient'):   
    from backbones.efficientnet.effnetv2_model import *
    import backbones.efficientnet.effnetv2_utils as effnet_util

    import tfu
    effnet_util.set_batchnorm(effnet_util.BatchNormalization)
    tfu.set_data_format('NHWC')
    tfu.set_dtype(tf.float32)

    intermediate_shape = [1, 8, 8, 1280]
    mod = get_model(model_name, include_top=False, pretrained=False, with_endpoints=False)
    
elif  model_name.startswith('resnet'):
    from options import FLAGS
    FLAGS.group_norm = False
    FLAGS.stride_test = 32
    FLAGS.stride_train = 32
    FLAGS.centered_stride = True
    bn = keras.layers.BatchNormalization
    
    import backbones.resnet
    class MyLayers(keras.layers.VersionAwareLayers):
        def __getattr__(self, name):
            if name == 'BatchNormalization':
                return bn
            return super().__getattr__(name)

    classname = f'ResNet{model_name[len("resnet"):]}'.replace('-', '_')
    mod = getattr(backbones.resnet, classname)(
        include_top=False, weights=None,
        input_shape=(None, None, 3), layers=MyLayers()) 
    intermediate_shape = [None, None, None, 2048]
    
mod.set_weights(mod_vars[:-4])

In [4]:
# save backbone model w/ full signature!

@tf.function()
def my_predict(my_prediction_inputs, **kwargs):
    prediction = mod(tf.expand_dims(my_prediction_inputs, axis=0), training=False)
    return {"prediction": prediction}

my_signatures = my_predict.get_concrete_function(
   my_prediction_inputs=tf.TensorSpec([256, 256, 3], dtype=tf.float32, name="image")
)

tf.saved_model.save(mod, bbone_name, signatures=my_signatures)

2022-01-03 15:22:04.278456: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mods/resnet152/bbone/assets


In [5]:
# save metrabs head only!

@tf.function()
def metr_head(my_prediction_inputs, **kwargs):
    #prediction = mod(my_prediction_inputs, training=False)
    pred2d, pred3d = model.crop_model.heatmap_heads(tf.cast(my_prediction_inputs, tf.float16), training=False)
    return {"prediction": pred2d}

metrab_sig = metr_head.get_concrete_function(
   my_prediction_inputs=tf.TensorSpec(intermediate_shape, dtype=tf.dtypes.float32, name="feature")
)


tf.saved_model.save(model.crop_model.heatmap_heads, head_name, signatures=metrab_sig)
if False:
    # confirm consistency briefly
    metrab_mod = tf.saved_model.load(head_name)
    pred2d_metrab, _ = metrab_mod(tf.cast(pred, tf.float16))
    assert(np.all(pred2d_metrab == pred2d)), 'saved model yields diff result'

INFO:tensorflow:Assets written to: mods/resnet152/metrab_head/assets


INFO:tensorflow:Assets written to: mods/resnet152/metrab_head/assets


In [2]:
print("run in regular env:")
print("python -m tf2onnx.convert --saved-model %s --output %s"%(bbone_name, onnx_name))

run in regular env:
python -m tf2onnx.convert --saved-model mods/effnet-l/bbone --output mods/effnet-l/bbone.onnx


In [ ]:
# test the full pipeline in its parts

%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import numpy as np

import time
import matplotlib.patches as patches
from PIL import Image

t3 = time.time()
img = Image.open('cpp/image.png')

t = time.time()
bbox = model.detector.predict_single_image(img)
t = time.time() - t

x, y, wd, ht, conf = bbox[0]
crop = np.array(img)[int(y):int(y+ht), int(x):int(x+wd)]
res = cv2.resize(crop, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)

Image.fromarray(res).save('cpp/crop.png')

inp = res.astype(np.float16)
inp /= 256.

t2 = time.time()
pred = mod(inp[np.newaxis,...], False)
t2 = time.time() - t2

t4 = time.time()
pred2d, pred3d = model.crop_model.heatmap_heads(tf.cast(pred, tf.float16), training=False)
#coords3d_abs = tfu3d.reconstruct_absolute(coords2d, coords3d, intrinsics)
t4 = time.time() - t4

t3 = time.time() - t3

rect = patches.Rectangle((x, y), wd, ht, linewidth=1, edgecolor='r', facecolor='none')
fig, ax = plt.subplots()
ax.imshow(img)
ax.add_patch(rect)
plt.show()

plt.imshow(crop)
plt.show()
plt.imshow(res)

xs = pred2d.numpy()[0, :, 0]
ys = pred2d.numpy()[0, :, 1]
plt.scatter(xs, ys)

print('det took %.2fms'%(t*1000))
print('pred took %.2fms'%(t2*1000))
print('metrabs took %.2fms'%(t4*1000))
print('total took %.2fms'%(t3*1000))
#print(pred)